# Introduction to Building LLM Agents with Tools and Tracing

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!pip install weave requests "mistralai[gcp]>=1.0.0"
```


In [1]:
# Global Configuration & Setup
import inspect
import json
import os
import requests
import weave # Must import weave before litellm for auto-patching
from enum import Enum
from pydantic import BaseModel, Field
from rich.pretty import pprint
from typing import Any, Callable, Dict, List, get_type_hints

# from mistralai_gcp import MistralGoogleCloud
from mistralai import Mistral

Define a model to use, as we are going to use tool calling you need a capable model like `mistral-large`

In [3]:

# Set the model name you want to use globally.

MISTRAL_LARGE_MODEL = "mistral-large-2411" # vertex name for the model
MISTRAL_MEDIUM_MODEL = "mistral-medium-latest"
MISTRAL_SMALL_MODEL = "mistral-small-2503"

client = Mistral(api_key=os.environ.get("MISTRAL_API_KEY"))

# region = os.environ.get("GOOGLE_CLOUD_REGION")
# project_id = os.environ.get("GOOGLE_CLOUD_PROJECT_ID")
# client = MistralGoogleCloud(region=region, project_id=project_id, timeout_ms=120_000)

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [4]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init('paris-woprkshop-2025')

/Users/tcapelle/work/phd_researcher/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


weave version 0.51.45 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: capecape.
View Weave data at https://wandb.ai/capecape/paris-woprkshop-2025/weave


## 1. Basic LLM Call with Mistral SDK

Let's start with a simple call to the LLM using/

![](images/01_trace.png)

In [5]:
# Define a simple message list (conversation history)
messages = [{"role": "user", "content": "Hello, LLM! How does an AI agent work?"}]

# Make the call
response = resp = client.chat.complete(
    model = MISTRAL_SMALL_MODEL,
    messages=messages,
)
# Print the response content
assistant_response = response.choices[0].message.content
print(f"LLM Response:\\n{assistant_response}")

# Click on the 🍩 linke below to see the trace in Weave 👇

🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b429-a314-7043-bce8-ea84afc3a2f0
LLM Response:\nHello! An AI agent is a software entity that can perceive its environment and take actions to achieve its goals. Here's a simplified breakdown of how an AI agent typically works:

1. **Perception**: The agent receives inputs from its environment through sensors or data sources. These inputs can be in various forms, such as text, images, audio, or other types of data.

2. **Processing**: The agent processes the received inputs using algorithms and models. This step involves several sub-processes:
   - **Understanding**: The agent interprets the inputs to understand the current state of the environment.
   - **Learning**: If the agent is designed to learn, it updates its internal models or parameters based on new data.
   - **Planning**: The agent decides on a course of action to achieve its goals. This could involve searching through possible actions and predicting their outcomes.

Because we imported `weave` before `litellm` and called `weave.init()`, the Mistral SDK call above was automatically traced. You can open your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [6]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: List[Dict[str, Any]], **kwargs) -> str:
    "Call a model with the given messages and kwargs."
    response = client.chat.complete(
        model=model_name,
        messages=messages,
        **kwargs
    )

    return response.choices[0].message

response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages)
print(response.content)

🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b42d-a10b-7e73-b4c1-062718005581
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b435-26eb-7671-b645-1d7b64d25a86
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b435-2b07-75b2-a87e-c8d8a4309881
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b438-f8f8-7921-9a05-61c9e373fb26
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b439-8e6f-70e3-80c4-683735ba8852
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b448-a265-74b3-a335-1887292fe54a🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b448-579c-7bc1-bfb2-bf16ccb6e925

🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b44a-9a45-7b62-bb55-67957946885f
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b450-e4a3-77e1-b9eb-c4ed9d08ec58
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b485-6fe3-70a2-94f6-e066b4c350fc
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b487-6772-

Hello! An AI agent is a system designed to perceive its environment and take actions to achieve specific goals. Here's a simplified breakdown of how an AI agent typically works:

### 1. **Perception**
   - **Sensors**: The agent uses sensors to gather data from its environment. These sensors can be physical (like cameras, microphones, or temperature sensors) or virtual (like data feeds from the internet).
   - **Data Processing**: The raw data collected by the sensors is processed to extract meaningful information. This might involve image recognition, natural language processing, or other forms of data analysis.

### 2. **Decision Making**
   - **Model**: The agent uses a model to make decisions. This model can be based on rules, machine learning algorithms, or a combination of both.
   - **Learning**: If the agent is based on machine learning, it may update its model over time based on new data and feedback. This is often done through techniques like reinforcement learning, supervise

![](images/02_nested_trace.png)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

Check the Mistral docs for function calling: https://docs.mistral.ai/capabilities/function_calling/

![](images/03_func_call.png)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [7]:
@weave.op 
def add_numbers(a: int, b: int) -> int:
    """Adds two numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [8]:
add_numbers(1, 2)

3

In [9]:
# this doesn't work...
call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages, tools=[add_numbers])

ValidationError: 2 validation errors for Unmarshaller
body.nullable[list[Tool]].0
  Input should be a valid dictionary or instance of Tool [type=model_type, input_value=<function add_numbers at 0x1307d6ac0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
body.Unset
  Input should be a valid dictionary or instance of Unset [type=model_type, input_value=[<function add_numbers at 0x1307d6ac0>], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

> We need to manually create the JSON schema describing this tool in a format that models *Mistral* understand.

In [10]:
# Manually define the tool schema
tool_add_numbers_schema = {
    "type": "function",
    "function": {
        "name": "add_numbers",
        "description": "Adds two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first number."
                },
                "b": {
                    "type": "integer",
                    "description": "The second number."
                }
            },
            "required": ["a", "b"]
        }
    }
}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [11]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that can use tools to answer questions."},
    {"role": "user", "content": "My lucky numbers are 77 and 11. What is their sum?"}]
response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages, tools=[tool_add_numbers_schema])
pprint(response)

🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b435-3c31-7c70-b110-c1fec92012b0


AssistantMessage(
│   content='',
│   tool_calls=[
│   │   ToolCall(
│   │   │   function=FunctionCall(name='add_numbers', arguments='{"a": 77, "b": 11}'),
│   │   │   id='kZv6pUpOz',
│   │   │   type=None,
│   │   │   index=0
│   │   )
│   ],
│   prefix=False,
│   role='assistant'
)

## Manual Tool Call
The LLM's response might contain a request to call our tool (`response.choices[0].message.tool_calls`) or it might respond directly (`response.choices[0].message.content`). If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [12]:
if response.tool_calls:
    print("LLM requested a tool call:")
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args_str = tool_call.function.arguments
        function_args = json.loads(function_args_str)
        print(f"  - Tool: {function_name}, Args: {function_args_str}")
        if function_name == "add_numbers":
            tool_result_content = add_numbers(**function_args)

print(tool_result_content)

LLM requested a tool call:
  - Tool: add_numbers, Args: {"a": 77, "b": 11}
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b435-6813-7c83-9de1-65830a4b7f3e
88


🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b437-59ea-7550-96a6-c22503443eb7
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b44a-1824-7411-897d-dbb26333bf40
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b44b-7b9f-7802-9ae9-f40b20b50313
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b44d-c9d7-7e91-aeb2-91154dabffb6
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b485-893c-7290-89ab-01364f287d12
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b48e-e221-7852-abb7-b81024bcae2f
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b492-1a1f-7971-b310-8b66365ace5f


We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [13]:
messages.append(response.model_dump())

In [14]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": str(tool_result_content)
})

In [15]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant that can use tools to answer questions.'},
 {'role': 'user',
  'content': 'My lucky numbers are 77 and 11. What is their sum?'},
 {'content': '',
  'tool_calls': [{'function': {'name': 'add_numbers',
     'arguments': '{"a": 77, "b": 11}'},
    'id': 'kZv6pUpOz',
    'type': None,
    'index': 0}],
  'prefix': False,
  'role': 'assistant'},
 {'tool_call_id': 'kZv6pUpOz',
  'role': 'tool',
  'name': 'add_numbers',
  'content': '88'}]

You should have a sequence of messages like this:


In [18]:
[m["role"] for m in messages]

['system', 'user', 'assistant', 'tool']

Now call the model again with the new messages and it will use the tool call result to answer the question

In [19]:
final_response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages)
print(final_response.content)

🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b436-5c07-7960-92c7-c2095c3dd573
The sum of 77 and 11 is 88.


## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [20]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a tool-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": inspect.getdoc(func).split("\\n")[0] if inspect.getdoc(func) else "",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    docstring = inspect.getdoc(func)
    param_descriptions = {}
    if docstring:
        args_section = False
        current_param = None
        for line in docstring.split('\\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped:
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" "): # Heuristic: end of args section
                     args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle List[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by List or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: OpenAI schema doesn't officially use default, but useful metadata

        schema["function"]["parameters"]["properties"][name] = param_schema
        if is_required:
            schema["function"]["parameters"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [21]:
tool_schema = generate_tool_schema(add_numbers)
pprint(tool_schema)

{
│   'type': 'function',
│   'function': {
│   │   'name': 'add_numbers',
│   │   'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
│   │   'parameters': {
│   │   │   'type': 'object',
│   │   │   'properties': {
│   │   │   │   'a': {'type': 'integer', 'description': ''},
│   │   │   │   'b': {'type': 'integer', 'description': ''}
│   │   │   },
│   │   │   'required': ['a', 'b']
│   │   }
│   }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [22]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [23]:
add_numbers = function_tool(add_numbers)
pprint(add_numbers.tool_schema)


{
│   'type': 'function',
│   'function': {
│   │   'name': 'add_numbers',
│   │   'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
│   │   'parameters': {
│   │   │   'type': 'object',
│   │   │   'properties': {
│   │   │   │   'a': {'type': 'integer', 'description': ''},
│   │   │   │   'b': {'type': 'integer', 'description': ''}
│   │   │   },
│   │   │   'required': ['a', 'b']
│   │   }
│   }
}

In [24]:
add_numbers.tool_schema

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': ''},
    'b': {'type': 'integer', 'description': ''}},
   'required': ['a', 'b']}}}

and call the tool =)

In [25]:
add_numbers(1, 2)

3

### 3.1 Real Example using an API based tool

Let's create a real example hitting an API: The pokemon API

In [27]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def get_pokemon_info(pokemon_name: str) -> str:
    """Fetches minimal information (name, ID, weight) for a specific Pokemon. Weight is in hectograms.

    Args:
        pokemon_name: The name or Pokedex ID of the Pokemon.
    """
    base_url = "https://pokeapi.co/api/v2/pokemon/"
    try:
        response = requests.get(f"{base_url}{pokemon_name.lower().strip()}")
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        info = {
            "name": data.get('name', 'Unknown').capitalize(),
            "id": data.get('id', -1),
            "weight": data.get('weight', -1) # Weight in hectograms
        }
        return info
    except Exception as e:
        return {"error": f"Error fetching pokemon {pokemon_name}: {str(e)}"}

In [29]:
get_pokemon_info.tool_schema

{'type': 'function',
 'function': {'name': 'get_pokemon_info',
  'description': 'Fetches minimal information (name, ID, weight) for a specific Pokemon. Weight is in hectograms.\n\nArgs:\n    pokemon_name: The name or Pokedex ID of the Pokemon.',
  'parameters': {'type': 'object',
   'properties': {'pokemon_name': {'type': 'string', 'description': ''}},
   'required': ['pokemon_name']}}}

In [30]:
get_pokemon_info("pikachu")

🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b438-52fe-7e10-a107-677affd7607d
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b438-db90-7150-ba99-c7b89c91a5b6
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b438-dbe6-7bd3-9465-f100c06e917e
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b44d-0634-7561-ae11-f89a3fc65594
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b48e-ea50-72d0-b4d0-11f3392ca3fd
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b490-7360-7cd0-870c-f03bdbfe122d
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b491-02d0-7c30-adc8-96ddcb649904


{'name': 'Pikachu', 'id': 25, 'weight': 60}

In [31]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that can use tools to answer questions."},
    {"role": "user", "content": "What is the weight of Pikachu?"}]

response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages, tools=[get_pokemon_info.tool_schema])
pprint(response)

🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b438-7d86-7251-a6b6-e1bd7298920a
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b438-f8a2-7ef3-a8e1-f23b962af356
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b449-cddb-7f00-8920-147bbbf91f7d
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b451-0a9c-7ae1-9390-87469ea2c103
🍩 https://wandb.ai/capecape/paris-woprkshop-2025/r/call/0196b485-f515-7bd1-a2d9-2269d0e88e4c


AssistantMessage(
│   content='',
│   tool_calls=[
│   │   ToolCall(
│   │   │   function=FunctionCall(name='get_pokemon_info', arguments='{"pokemon_name": "Pikachu"}'),
│   │   │   id='TiekqiK6c',
│   │   │   type=None,
│   │   │   index=0
│   │   )
│   ],
│   prefix=False,
│   role='assistant'
)

Let's create some helper functions to perform the tool calls

In [69]:
from mistralai import ToolCall

def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], tool_calls: list[ToolCall]) -> list[dict]:
    "Perform the tool calls and return the messages with the tool call results"
    messages = []
    for tool_call in tool_calls:
        print(f"Performing tool call: {tool_call.function.name}")
        print(f"  - Args: {tool_call.function.arguments}")
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        tool = get_tool(tools, function_name)
        tool_response = tool(**function_args)
        print(f"  - Response: {tool_response}")
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "content": str(tool_response),
        })
    return messages

In [70]:

# add the tool call result to the messages
messages.append(response.model_dump())
messages.extend(perform_tool_calls(tools=[get_pokemon_info], tool_calls=response.tool_calls))

final_response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages)
print(final_response.content)


Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "Pikachu"}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
Pikachu weighs 60 pounds.


Let's wrap this in a function and add a few more tools.

In [71]:
@weave.op
def pokedex(pokemon_question: str) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant that can use tools to answer questions."},
        {"role": "user", "content": pokemon_question}]

    # call model with tools
    response = call_model(
        model_name=MISTRAL_SMALL_MODEL, 
        messages=messages, 
        tools=[get_pokemon_info.tool_schema, 
            add_numbers.tool_schema])

    # add the response to the messages
    messages.append(response.model_dump())

    # perform the tool calls
    messages.extend(perform_tool_calls(tools=[get_pokemon_info, add_numbers], tool_calls=response.tool_calls))

    final_response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages)
    return final_response.content

In [72]:
pokedex("What is the combined weight of Ash's first 3 pokemons?")

Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "pikachu"}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "caterpie"}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "butterfree"}
  - Response: {'name': 'Butterfree', 'id': 12, 'weight': 320}


"The combined weight of Ash's first 3 Pokémon is 409 units."

![](images/04_pokedex.png)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](images/05_agent.png)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [73]:
@weave.op
def pokedex_loop(pokemon_question: str, max_turns: int = 4, tools = [get_pokemon_info, add_numbers]) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant that can use tools to answer questions."},
        {"role": "user", "content": pokemon_question}]

    
    for turn in range(max_turns):
        print(f"--- Agent Loop Turn {turn + 1}/{max_turns} ---")

        # call model with tools
        response = call_model(
            model_name=MISTRAL_SMALL_MODEL, 
            messages=messages, 
            tools=[t.tool_schema for t in tools])

        # add the response to the messages
        messages.append(response.model_dump())

        # if the LLM requested tool calls, perform them
        if response.tool_calls:
            print("LLM requested tool calls:")
            # perform the tool calls
            tool_outputs = perform_tool_calls(tools=[get_pokemon_info, add_numbers], tool_calls=response.tool_calls)
            messages.extend(tool_outputs)
        # LLM gave content response
        elif response.content:
            print(f"LLM content response: {response.content}")
            return response.content
        else:
            print("LLM response had neither content nor tool calls. Stopping loop.")
            break

In [74]:
pokedex_loop("What is the combined weight of Ash's first 3 pokemons?")

--- Agent Loop Turn 1/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "pikachu"}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "caterpie"}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "pidgey"}
  - Response: {'name': 'Pidgey', 'id': 16, 'weight': 18}
--- Agent Loop Turn 2/4 ---
LLM requested tool calls:
Performing tool call: add_numbers
  - Args: {"a": 60, "b": 29}
  - Response: 89
Performing tool call: add_numbers
  - Args: {"a": 60, "b": 18}
  - Response: 78
--- Agent Loop Turn 3/4 ---
LLM content response: The combined weight of Ash's first 3 pokemons is 167 hectograms.


"The combined weight of Ash's first 3 pokemons is 167 hectograms."

In [75]:
pokedex_loop("Name Ash's first 3 pokemons and calculate their combined weight?")

--- Agent Loop Turn 1/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "pikachu"}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "caterpie"}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "pidgey"}
  - Response: {'name': 'Pidgey', 'id': 16, 'weight': 18}
--- Agent Loop Turn 2/4 ---
LLM content response: Ash's first three Pokémon are Pikachu, Caterpie, and Pidgey. Their combined weight is 107 hectograms.


"Ash's first three Pokémon are Pikachu, Caterpie, and Pidgey. Their combined weight is 107 hectograms."

![](images/06_pokedex_loop.png)

This one is kind of an hallucination, I was expecting snorlax to be the heaviest pokemon.

In [76]:
pokedex_loop("Which is the heaviest pokemon?")

--- Agent Loop Turn 1/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "1"}
  - Response: {'name': 'Bulbasaur', 'id': 1, 'weight': 69}
--- Agent Loop Turn 2/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "2"}
  - Response: {'name': 'Ivysaur', 'id': 2, 'weight': 130}
--- Agent Loop Turn 3/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "3"}
  - Response: {'name': 'Venusaur', 'id': 3, 'weight': 1000}
--- Agent Loop Turn 4/4 ---
LLM content response: The heaviest pokemon is Venusaur, with a weight of 1000 hectograms.


'The heaviest pokemon is Venusaur, with a weight of 1000 hectograms.'

6. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [77]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: List[Dict[str, Any]] = Field(default_factory=list)
    step: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

In [82]:
class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: List[Callable]):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
    
    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step = state.step + 1
        messages = state.messages
        final_assistant_content = None
        try:
            # call model with tools
            response = call_model(
                model_name=self.model_name, 
                messages=messages, 
                tools=[t.tool_schema for t in self.tools])

            # add the response to the messages
            messages.append(response.model_dump())

            # if the LLM requested tool calls, perform them
            if response.tool_calls:
                print("LLM requested tool calls:")
                # perform the tool calls
                tool_outputs = perform_tool_calls(tools=[get_pokemon_info, add_numbers], tool_calls=response.tool_calls)
                messages.extend(tool_outputs)

            # LLM gave content response
            else:
                messages.append(response.model_dump())
                final_assistant_content = response.content
        except Exception as e:
            print(f"ERROR in Agent Step: {e}")
            # Add an error message to history to indicate failure
            messages.append({"role": "assistant", "content": f"Agent error in step: {str(e)}"})
            final_assistant_content = f"Agent error in step {step}: {str(e)}"
        return AgentState(messages=messages, step=step, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10) -> AgentState:
        state = AgentState(messages=[{"role": "user", "content": user_prompt}])
        for _ in range(max_turns):
            print(f"--- Agent Loop Turn {state.step}/{max_turns} ---")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state


![](images/07_simple_agent.png)

In [ ]:
agent = SimpleAgent(
    model_name=MISTRAL_SMALL_MODEL,
    system_message="You are a helpful assistant that can use tools to answer questions.",
    tools=[get_pokemon_info, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")


--- Agent Loop Turn 0/10 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "Pikachu"}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "Caterpie"}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "Pidgey"}
  - Response: {'name': 'Pidgey', 'id': 16, 'weight': 18}
--- Agent Loop Turn 1/10 ---
LLM requested tool calls:
Performing tool call: add_numbers
  - Args: {"a": 60, "b": 29}
  - Response: 89
Performing tool call: add_numbers
  - Args: {"a": 60, "b": 18}
  - Response: 78
--- Agent Loop Turn 2/10 ---
The combined weight of Ash's first 3 pokemons is 89 + 78 = 167 hectograms.
